 1. Configuração Inicial e Verificação de Ambiente (Adaptado para MacBook M3)

In [1]:

import os
import torch
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer

# Verificação de ambiente para Apple Silicon (M-series)
if torch.backends.mps.is_available():
    device = "mps"
    print("🎉 Apple Silicon (MPS) detectado! Usando aceleração via Metal Performance Shaders.")
elif torch.cuda.is_available():
    device = "cuda"
    print(f"🎉 GPU detectada! Usando: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    print("⚠️ Nenhuma GPU ou MPS detectado. O treinamento será executado em CPU.")
print(f"Dispositivo selecionado para PyTorch: {device}\n")

# Para garantir a reprodutibilidade
def set_seed(seed: int = 42):
    import random
    import numpy as np
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
set_seed(42)

/Users/luizviana/tech_chall_3_v2/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🎉 Apple Silicon (MPS) detectado! Usando aceleração via Metal Performance Shaders.
Dispositivo selecionado para PyTorch: mps



2. Carregamento e Amostragem do Dataset

In [2]:
# Defina o caminho para o seu arquivo de dados
data_file_path = 'trn.json'

print(f"Carregando o dataset '{data_file_path}'...")
try:
    
    df = pd.read_json(data_file_path, lines=True)
    print("Dataset carregado com sucesso!")
except Exception as e:
    print(f"Erro ao carregar o dataset: {e}")
    print("Verifique o caminho do arquivo e o formato. Se não for JSON Lines, remova `lines=True` ou ajuste o método de leitura.")
    # Crie um DataFrame de exemplo para continuar o notebook se houver erro no carregamento real
    data = {
        'title': ['Boneca Interativa Bebê', 'Kit Maquiagem Infantil Segura', 'Livro de Contos Mágicos', 'Robô de Brinquedo Programável', 'Carrinho de Controle Remoto'],
        'content': [
            'Minha filha adorou esta boneca! Ela interage, ri e até pede comida. Feita com materiais super seguros para crianças pequenas, sem peças que se soltam. Perfeita para idades de 2 a 5 anos.',
            'Cores vibrantes e fácil de aplicar. Minha neta se sente uma estrela! Testado dermatologicamente e totalmente seguro, sem causar alergias. Vem com um estojo prático para guardar.',
            'Histórias cativantes e ilustrações lindas. A cada página, uma nova aventura. Ideal para leitura antes de dormir, estimula a imaginação. Capa dura e páginas resistentes.',
            'Este robô é incrível! Fácil de programar com o aplicativo e executa comandos complexos. Horas de diversão para crianças maiores. Ótimo para introduzir conceitos de codificação.',
            'O carrinho tem uma velocidade impressionante e ótima durabilidade. Bateria de longa duração e controle preciso. Excelente presente para entusiastas de carros de todas as idades.'
        ]
    }
    df = pd.DataFrame(data)
    print("Usando um DataFrame de exemplo para demonstração.")


# Selecionar as colunas 'title' e 'content'
df_filtered = df[['title', 'content']]

# Criar uma amostra aleatória de 100.000 observações
sample_size = min(100000, len(df_filtered))
df_sample = df_filtered.sample(n=sample_size, random_state=42)

print(f"\nDataset original carregado: {len(df)} observações")
print(f"Dataset amostrado: {len(df_sample)} observações")
print("\nPrimeiras 5 linhas do dataset amostrado:")
print(df_sample.head())
print("\nInformações do dataset amostrado:")
df_sample.info()

Carregando o dataset 'trn.json'...
Dataset carregado com sucesso!

Dataset original carregado: 2248619 observações
Dataset amostrado: 100000 observações

Primeiras 5 linhas do dataset amostrado:
                                                     title  \
2183177         Yellow Box Women's Yulisa Flip Flop Sandal   
1878879                   Nine West Women's Eastnor Sandal   
1140718  Roma Costume Women's 1 Piece Santa's Envy-Red ...   
1082826  1 Box of 100 Sony 1.55v Silver Oxide Watch Bat...   
54788    Winning Angels: The 7 Fundamentals of Early St...   

                                                   content  
2183177                                                     
1878879                                                     
1140718                                                     
1082826  100 Sony 377 Watch Batteries Silver Oxide, Mer...  
54788    "Winning Angels is a superbly organised and in...  

Informações do dataset amostrado:
<class 'pandas.core.frame.DataF

3. Preparação dos Dados para Fine-tuning

In [ ]:

from datasets import Dataset

def formatting_prompts_func(examples):
    output_texts = []
    for i in range(len(examples['content'])):
        
        content = str(examples['content'][i] if examples['content'][i] is not None else "").strip()
        title = str(examples['title'][i] if examples['title'][i] is not None else "").strip()
        if not content or not title:
            continue
        prompt = (
            f"<s>[INST] Analise o seguinte comentário/descrição de produto: {content} [/INST] "
            f"O título do produto que melhor corresponde a essa descrição é: {title}</s>"
        )
        output_texts.append(prompt)
    return {"text": output_texts}

# Converte o DataFrame para um objeto Dataset do Hugging Face
# print("Convertendo DataFrame para Dataset...")
dataset = Dataset.from_pandas(df_sample)

# Dividir em conjuntos de treinamento e validação
print("Dividindo dataset em treinamento e validação...")
train_test_split = dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

print("Aplicando função de formatação e removendo colunas desnecessárias nos datasets...")
train_dataset = train_dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=train_dataset.column_names
)
eval_dataset = eval_dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=eval_dataset.column_names
)

# Filtra quaisquer exemplos que possam ter resultado em 'text' vazio após a formatação
print("Filtrando exemplos vazios após a formatação...")
train_dataset = train_dataset.filter(lambda x: len(x['text']) > 0)
eval_dataset = eval_dataset.filter(lambda x: len(x['text']) > 0)

print(f"Dataset de treinamento formatado e filtrado: {len(train_dataset)} exemplos")
print(f"Dataset de validação formatado e filtrado: {len(eval_dataset)} exemplos")

# Exibir um exemplo formatado do dataset já processado
print("\nExemplo de prompt formatado no dataset final (train_dataset[0]['text'][0]):")
if len(train_dataset) > 0:
    print(train_dataset[0]['text'][0])
else:
    print("O dataset de treinamento está vazio após formatação e filtragem. Verifique seus dados originais.")


Dividindo dataset em treinamento e validação...
Aplicando função de formatação e removendo colunas desnecessárias nos datasets...


Map: 100%|██████████| 5000/5000 [00:00<00:00, 265465.64 examples/s]


Filtrando exemplos vazios após a formatação...


Filter: 100%|██████████| 3017/3017 [00:00<00:00, 773437.76 examples/s]

Dataset de treinamento formatado e filtrado: 58857 exemplos
Dataset de validação formatado e filtrado: 3017 exemplos

Exemplo de prompt formatado no dataset final (train_dataset[0]['text'][0]):
<


4. Carregamento do Modelo Base e Tokenizador

In [10]:

# Define o ID do modelo base
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print(f"Carregando tokenizador para o modelo '{model_id}'...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Necessário para treinamento eficiente de LLMs


tokenizer.model_max_length = 512

print(f"\nCarregando modelo base '{model_id}' para {device}...\n")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16 if device == "mps" else torch.float32,
    device_map="auto"
)

# Configurações do modelo para fine-tuning
model.config.use_cache = False
model.config.pretraining_tp = 1
print("Modelo e tokenizador carregados com sucesso!")


Carregando tokenizador para o modelo 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'...

Carregando modelo base 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' para mps...



`torch_dtype` is deprecated! Use `dtype` instead!


Modelo e tokenizador carregados com sucesso!


5. Configuração do LoRA (PEFT)

In [12]:
print("Configurando LoRA (Low-Rank Adaptation)...")
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

# Aplica a configuração LoRA ao modelo
model = get_peft_model(model, lora_config)

print("Modelo configurado com adaptadores LoRA:")
model.print_trainable_parameters()

Configurando LoRA (Low-Rank Adaptation)...
Modelo configurado com adaptadores LoRA:
trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044


/Users/luizviana/tech_chall_3_v2/env/lib/python3.12/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/Users/luizviana/tech_chall_3_v2/env/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


6. Definição dos Argumentos de Treinamento

In [13]:

output_dir = "./fine_tuned_llama"

print("Definindo argumentos de treinamento...")
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1, # Uma única época de treinamento
    per_device_train_batch_size=2, # REDUZIDO para 2 para economia de memória
    gradient_accumulation_steps=4, # AUMENTADO para 4 para manter o effective_batch_size
    learning_rate=2e-4,
    logging_steps=100,
    save_steps=1000,
    eval_strategy="steps",
    eval_steps=1000,
    report_to="none",
    optim="adamw_torch",
    fp16=False,
    bf16=True if device == "mps" else False,
    seed=42,
)

print("Argumentos de treinamento definidos.")

Definindo argumentos de treinamento...
Argumentos de treinamento definidos.


7. Inicialização do SFTTrainer

In [14]:

print("Inicializando SFTTrainer...")

# Data Collator: Usado para agrupar e preencher os exemplos de dados em um batch.
# Ele usará o tokenizer que já configuramos com model_max_length.
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset, # Agora o dataset já está pré-formatado e contém a coluna 'text'
    eval_dataset=eval_dataset,
    peft_config=lora_config,
    # REMOVIDO: 'formatting_func' não é mais necessário aqui, pois o dataset já foi formatado na Célula 3.
    data_collator=data_collator,
    args=training_arguments,
    # REMOVIDO: 'max_seq_length' também foi removido, pois o tokenizer e o data_collator devem lidar com isso.
)

print("SFTTrainer inicializado.")

Inicializando SFTTrainer...


Truncating eval dataset: 100%|██████████| 3017/3017 [00:00<00:00, 1708182.39 examples/s]

SFTTrainer inicializado.


8. Início do Treinamento

In [8]:
print("Iniciando o treinamento do modelo...")
trainer.train()
print("Treinamento concluído!")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Iniciando o treinamento do modelo...


/Users/luizviana/tech_chall_3_v2/env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1000,1.865200,1.897628,1.918301,1788856.000000,0.609811
2000,1.907400,1.886621,1.897021,3576213.000000,0.611650
3000,1.850400,1.880466,1.874104,5372447.000000,0.612472
4000,1.839200,1.875961,1.869433,7158034.000000,0.613233
5000,1.867800,1.872083,1.887396,8939609.000000,0.613708
6000,1.885700,1.868985,1.876670,10730210.000000,0.614225
7000,1.850400,1.867453,1.875877,12552728.000000,0.614415


/Users/luizviana/tech_chall_3_v2/env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/luizviana/tech_chall_3_v2/env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/luizviana/tech_chall_3_v2/env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/luizviana/tech_chall_3_v2/env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Use

Treinamento concluído!


Célula 9 - Teste do modelo tunado

In [2]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel


# Detecção de dispositivo como na primeira célula
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
print(f"Carregando modelo base '{model_id}' para inferência...")

# -- CORREÇÃO PRINCIPAL AQUI --
# 1. Carregar o modelo base SEM device_map="auto" para evitar wrappers adicionais.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16 if device == "mps" else torch.float32,
    # REMOVIDO: device_map="auto" aqui.
)
# O modelo será carregado na CPU por padrão (ou o dispositivo padrão do PyTorch para o dtype).

print(f"Carregando tokenizer '{model_id}'...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.model_max_length = 512
print("Tokenizer carregado e configurado.")

base_output_dir = "./fine_tuned_llama"
fine_tuned_model_path = base_output_dir

# Lógica para encontrar o último checkpoint (a mais robusta)
if os.path.exists(base_output_dir):
    checkpoints = [
        os.path.join(base_output_dir, d)
        for d in os.listdir(base_output_dir)
        if os.path.isdir(os.path.join(base_output_dir, d)) and d.startswith("checkpoint-")
    ]
    if checkpoints:
        # Encontra o checkpoint com o maior número de passos (o mais recente)
        fine_tuned_model_path = max(checkpoints, key=lambda cp: int(cp.split('-')[-1]))
        print(f"Último checkpoint encontrado: {fine_tuned_model_path}")
    else:
        # Se nenhum subdiretório 'checkpoint-XXXX' for encontrado, tentamos o diretório base.
        print(f"Nenhum subdiretório 'checkpoint-XXXX' encontrado em '{base_output_dir}'. Tentando carregar diretamente de '{base_output_dir}'.")
        if not os.path.exists(os.path.join(base_output_dir, "adapter_config.json")):
             print(f"Erro: 'adapter_config.json' não encontrado em '{base_output_dir}'. Verifique o diretório de saída do treinamento.")
             raise FileNotFoundError(f"Não foi possível encontrar o modelo LoRA em '{fine_tuned_model_path}'")
else:
    print(f"Diretório de saída '{base_output_dir}' não encontrado. Verifique se o treinamento foi concluído e salvou os arquivos.")
    raise FileNotFoundError(f"O diretório '{base_output_dir}' não existe. Certifique-se de que o treinamento foi executado.")


print(f"Carregando adaptadores LoRA de '{fine_tuned_model_path}'...\n")
# Carrega os adaptadores PEFT e os anexa ao modelo base.
# 'model' aqui é o AutoModelForCausalLM puro, sem wrappers de device_map.
model = PeftModel.from_pretrained(model, fine_tuned_model_path)

# -- SEGUNDA CORREÇÃO --
# 2. Agora que o PeftModel foi criado corretamente (envolvendo o modelo base puro),
# movemos TODO o PeftModel para o dispositivo.
model.to(device)

# A linha de merge_and_unload() continua removida, pois não é a causa raiz do erro e pode
# ser problemática em outros cenários.
# print("Mesclando adaptadores LoRA com o modelo base...")
# model = model.merge_and_unload()

model.eval() # Coloca o modelo em modo de avaliação

print("Modelo fine-tuned (com adaptadores LoRA) pronto para inferência!\n")

# Cria o pipeline de geração de texto
text_generator = pipeline(
    "text-generation",
    model=model, # Passamos o PeftModel diretamente
    tokenizer=tokenizer,
    # O device não precisa ser explicitado aqui, pois o modelo já está no dispositivo correto.
    # Se quiser ser explícito, pode usar: device=0 if device == "cuda" else -1
)


def generate_product_title(description: str, max_new_tokens: int = 20) -> str:
    # O prompt deve ser exatamente como foi formatado para o treinamento,
    # até o ponto em que esperamos a geração do modelo.
    # Exemplo de treinamento: <s>[INST] {content} [/INST] O título do produto que melhor corresponde a essa descrição é: {title}</s>
    inference_prompt = (
        f"<s>[INST] Analise o seguinte comentário/descrição de produto: {description} [/INST] "
        f"O título do produto que melhor corresponde a essa descrição é: "
    )

    # Gerar a resposta
    generated_output = text_generator(
        inference_prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        return_full_text=False # Esta opção é crucial para o pipeline retornar apenas o texto gerado após o prompt
    )

    # O pipeline com return_full_text=False já retorna apenas a parte gerada.
    # Precisamos apenas limpar os tokens especiais residuais.
    if generated_output and len(generated_output) > 0:
        generated_title = generated_output[0]['generated_text'].strip()
    else:
        return "Não foi possível gerar um título." # Em caso de falha na geração

    # Remove quaisquer tokens de parada ou marcações do prompt que possam ter sido gerados
    # ou que o modelo "ecoou" e não foram limpos automaticamente.
    # Usamos split para garantir que pegamos apenas a parte antes do primeiro token de parada.
    generated_title = generated_title.split("</s>")[0].strip()
    generated_title = generated_title.split("[/INST]")[0].strip() # Para o caso de uma geração incompleta ou errada

    return generated_title

# --- Exemplos de Teste (para validação inicial da Célula 9) ---
print("\n--- Testes Rápidos do Modelo (após Célula 9) ---\n")

desc1 = "Fone de ouvido sem fio com cancelamento de ruído e bateria de longa duração."
title1 = generate_product_title(desc1)
print(f"Descrição: '{desc1}'\nTítulo Gerado: '{title1}'\n")

desc2 = "Laptop leve e potente para trabalho e jogos, com tela de 15 polegadas e processador de última geração."
title2 = generate_product_title(desc2)
print(f"Descrição: '{desc2}'\nTítulo Gerado: '{title2}'\n")



Usando dispositivo: mps
Carregando modelo base 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' para inferência...


`torch_dtype` is deprecated! Use `dtype` instead!


Carregando tokenizer 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'...
Tokenizer carregado e configurado.
Último checkpoint encontrado: ./fine_tuned_llama/checkpoint-7358
Carregando adaptadores LoRA de './fine_tuned_llama/checkpoint-7358'...



Device set to use mps:0


Modelo fine-tuned (com adaptadores LoRA) pronto para inferência!


--- Testes Rápidos do Modelo (após Célula 9) ---

Descrição: 'Fone de ouvido sem fio com cancelamento de ruído e bateria de longa duração.'
Título Gerado: '2 in 1 Battery / Loudspeaker Earphone - B&amp;O'

Descrição: 'Laptop leve e potente para trabalho e jogos, com tela de 15 polegadas e processador de última geração.'
Título Gerado: '2012 Acer Aspire 7755T-7032G'



Célula 10 - Teste interativo para resposta de títulos de produtos a partir das características 

In [ ]:

import ipywidgets as widgets
from IPython.display import display, clear_output

# Cria o widget de entrada de texto para a descrição do produto
description_input = widgets.Textarea(
    value='',
    placeholder='Digite a descrição do produto aqui...',
    description='Descrição:',
    disabled=False,
    layout=widgets.Layout(width='auto', height='100px') # Ajusta o layout para ocupar a largura total
)

# Cria um botão para gerar o título
generate_button = widgets.Button(
    description='Gerar Título',
    disabled=False,
    button_style='success', # Estilo visual do botão (verde)
    tooltip='Clique para gerar o título do produto',
    icon='magic' # Ícone visual no botão
)

# Cria um widget de saída para exibir o resultado e mensagens de carregamento
output_widget = widgets.Output()

# Função que será chamada quando o botão for clicado
def on_generate_button_clicked(b):
    with output_widget:
        clear_output()
        description = description_input.value.strip()
        
        if not description:
            print("Por favor, digite uma descrição para o produto.")
            return

        print("Gerando título... Isso pode levar alguns segundos. Por favor, aguarde.")
        try:
            generated_title = generate_product_title(description)
            print(f"\n--- Resultado da Geração ---")
            print(f"Descrição fornecida:\n{description}")
            print(f"\nTítulo Gerado:\n{generated_title}")
            print(f"\n--- Fim do Resultado ---")
        except Exception as e:
            print(f"Ocorreu um erro durante a geração do título: {e}")
            print("Verifique se o modelo foi carregado corretamente na Célula 9 e se a função generate_product_title está acessível.")

# Associa a função ao evento de clique do botão
generate_button.on_click(on_generate_button_clicked)

# Exibe os widgets na saída da célula
print("--- Interface Interativa para Testar o Modelo Tunado ---")
display(description_input, generate_button, output_widget)

print("\nInstruções de Uso:")
print("1. Digite a descrição completa do produto na caixa de texto acima.")
print("2. Clique no botão 'Gerar Título'.")
print("3. O título gerado pelo seu modelo fine-tuned aparecerá na área de resultados abaixo do botão.")
print("4. Para uma nova geração, apague ou edite a descrição e clique novamente.")



ipywidgets já está instalado.
--- Interface Interativa para Testar o Modelo Tunado ---


Textarea(value='', description='Descrição:', layout=Layout(height='100px', width='auto'), placeholder='Digite …

Button(button_style='success', description='Gerar Título', icon='magic', style=ButtonStyle(), tooltip='Clique …

Output()


Instruções de Uso:
1. Digite a descrição completa do produto na caixa de texto acima.
2. Clique no botão 'Gerar Título'.
3. O título gerado pelo seu modelo fine-tuned aparecerá na área de resultados abaixo do botão.
4. Para uma nova geração, apague ou edite a descrição e clique novamente.
